In [28]:
# Importing necessary libraries
import pandas
import newspaper
import re
import simhash

In [2]:
# Reading in the extracted content
extracted = pandas.read_csv("../data/url_info.csv", encoding = "iso8859")
extracted.head()

,url,status_code,title,content,content_hash
0,https://z94radio.iheart.com/content/2018-01-23...,200.0,Z-94 - Minot's Hit Music Station,"<html class=""no-touchevents"">\r\n<head>\r\n<me...",NaN
1,https://yournews.com/2018/01/23/289253/one-per...,200.0,Investigators probe motive in deadly Kentucky ...,"<html lang=""en-US"" prefix=""og: http://ogp.me/n...",NaN
2,https://xl93.iheart.com/content/2018-01-23-mul...,200.0,Two People Killed In Kentucky High School Shoo...,"<html class=""no-touchevents"">\r\n<head>\r\n<me...",NaN
3,https://www.zerohedge.com/news/2018-01-23/leas...,200.0,"Two Killed, 12 Wounded In Kentucky High School...","<html dir=""ltr"" lang=""en"" prefix=""content: htt...",NaN
4,https://www.zerohedge.com/news/2018-01-23/leas...,200.0,"Two Killed, 12 Wounded In Kentucky High School...","<html dir=""ltr"" lang=""en"" prefix=""content: htt...",NaN


In [3]:
# Function to extract the text from the html of each article
def convert_text(html):
    try:
        return newspaper.fulltext(html)
    except:
        return ""

In [4]:
extracted["text"] = extracted.content.apply(convert_text)

In [5]:
extracted.head()

,url,status_code,title,content,content_hash,text
0,https://z94radio.iheart.com/content/2018-01-23...,200.0,Z-94 - Minot's Hit Music Station,"<html class=""no-touchevents"">\r\n<head>\r\n<me...",NaN,Content Goes Here\n\nAdd to Playlist Create Ne...
1,https://yournews.com/2018/01/23/289253/one-per...,200.0,Investigators probe motive in deadly Kentucky ...,"<html lang=""en-US"" prefix=""og: http://ogp.me/n...",NaN,(Reuters)  Investigators in western Kentucky ...
2,https://xl93.iheart.com/content/2018-01-23-mul...,200.0,Two People Killed In Kentucky High School Shoo...,"<html class=""no-touchevents"">\r\n<head>\r\n<me...",NaN,At least two people have died and 19 people we...
3,https://www.zerohedge.com/news/2018-01-23/leas...,200.0,"Two Killed, 12 Wounded In Kentucky High School...","<html dir=""ltr"" lang=""en"" prefix=""content: htt...",NaN,Update ( 2 pm ET): The death toll in today's s...
4,https://www.zerohedge.com/news/2018-01-23/leas...,200.0,"Two Killed, 12 Wounded In Kentucky High School...","<html dir=""ltr"" lang=""en"" prefix=""content: htt...",NaN,Update ( 2 pm ET): The death toll in today's s...


In [20]:
space_re = re.compile("[\s]{2,}")
char_re = re.compile("([A-Za-z0-9]+)[^A-Za-z0-9\s]+([A-Za-z0-9]+)")
right_re = re.compile("([A-Za-z0-9]+)[^A-Za-z0-9\s]+")
left_re = re.compile("[^A-Za-z0-9\s]+([A-Za-z0-9]+)")
center_re = re.compile("\s*[^A-Za-z0-9\s]+\s*")
url_re = re.compile("https{0,1}://[^\s]+")
url2_re = re.compile("[a-z0-9\.]+\.[a-z0-9\.]+/[^\s]*")

def normalize_text(text):
    # Converting text to lowercase
    text = text.lower()
    
    # Removing URLs
    text = url_re.sub(" ", text)
    text = url2_re.sub(" ", text)
    
    # Removing non-alphanumeric characters
    text = char_re.sub("\g<1>\g<2>", text)
    text = right_re.sub("\g<1>", text)
    text = left_re.sub("\g<1>", text)  
    text = center_re.sub(" ", text)
    
    # Removing multiple spacing characters
    text = space_re.sub(" ", text)
    
    # Stripping leading and trailing spaces
    text = text.strip()
    
    return text

In [21]:
extracted["norm_text"] = extracted.text.apply(normalize_text)

In [22]:
extracted.head()

,url,status_code,title,content,content_hash,text,norm_text
0,https://z94radio.iheart.com/content/2018-01-23...,200.0,Z-94 - Minot's Hit Music Station,"<html class=""no-touchevents"">\r\n<head>\r\n<me...",NaN,Content Goes Here\n\nAdd to Playlist Create Ne...,content goes here add to playlist create new p...
1,https://yournews.com/2018/01/23/289253/one-per...,200.0,Investigators probe motive in deadly Kentucky ...,"<html lang=""en-US"" prefix=""og: http://ogp.me/n...",NaN,(Reuters)  Investigators in western Kentucky ...,reuters investigators in western kentucky on w...
2,https://xl93.iheart.com/content/2018-01-23-mul...,200.0,Two People Killed In Kentucky High School Shoo...,"<html class=""no-touchevents"">\r\n<head>\r\n<me...",NaN,At least two people have died and 19 people we...,at least two people have died and 19 people we...
3,https://www.zerohedge.com/news/2018-01-23/leas...,200.0,"Two Killed, 12 Wounded In Kentucky High School...","<html dir=""ltr"" lang=""en"" prefix=""content: htt...",NaN,Update ( 2 pm ET): The death toll in today's s...,update 2 pm et the death toll in todays shooti...
4,https://www.zerohedge.com/news/2018-01-23/leas...,200.0,"Two Killed, 12 Wounded In Kentucky High School...","<html dir=""ltr"" lang=""en"" prefix=""content: htt...",NaN,Update ( 2 pm ET): The death toll in today's s...,update 2 pm et the death toll in todays shooti...


In [25]:
extracted.iloc[48].norm_text

'benton ky ap a 15yearold student opened fire with a handgun inside his rural kentucky high school tuesday killing two classmates injuring 17 others and sending hundreds fleeing for safety police were seen leading a teenager away in handcuffs it was the nations first fatal school shooting of 2018 the shooter will be charged with murder and attempted murder kentucky state police lt michael webb said he was apprehended by the sheriffs department here on site at the school thankfully before any more lives could be taken webb said police did not release any identities nor did they describe a motive webb said detectives are looking into his home and background students ran for their lives as the student began shooting inside an atrium just before morning classes would have begun at marshall county high school many jumped into cars or ran down the highway some not stopping until they reached a mcdonalds restaurant more than a mile away parents left their cars on both sides of an adjacent roa

In [29]:
def get_features(s):
    width = 3
    s = s.lower()
    s = re.sub(r'[^\w]+', '', s)
    return [s[i:i + width] for i in range(max(len(s) - width + 1, 1))]

In [30]:
extracted.content_hash = extracted.norm_text.apply(lambda X: simhash.Simhash(get_features(X)).value)
extracted.head()

,url,status_code,title,content,content_hash,text,norm_text
0,https://z94radio.iheart.com/content/2018-01-23...,200.0,Z-94 - Minot's Hit Music Station,"<html class=""no-touchevents"">\r\n<head>\r\n<me...",17127191963060099840,Content Goes Here\n\nAdd to Playlist Create Ne...,content goes here add to playlist create new p...
1,https://yournews.com/2018/01/23/289253/one-per...,200.0,Investigators probe motive in deadly Kentucky ...,"<html lang=""en-US"" prefix=""og: http://ogp.me/n...",17090541445091345736,(Reuters)  Investigators in western Kentucky ...,reuters investigators in western kentucky on w...
2,https://xl93.iheart.com/content/2018-01-23-mul...,200.0,Two People Killed In Kentucky High School Shoo...,"<html class=""no-touchevents"">\r\n<head>\r\n<me...",9804137863708496329,At least two people have died and 19 people we...,at least two people have died and 19 people we...
3,https://www.zerohedge.com/news/2018-01-23/leas...,200.0,"Two Killed, 12 Wounded In Kentucky High School...","<html dir=""ltr"" lang=""en"" prefix=""content: htt...",400062519250097478,Update ( 2 pm ET): The death toll in today's s...,update 2 pm et the death toll in todays shooti...
4,https://www.zerohedge.com/news/2018-01-23/leas...,200.0,"Two Killed, 12 Wounded In Kentucky High School...","<html dir=""ltr"" lang=""en"" prefix=""content: htt...",400062519250097478,Update ( 2 pm ET): The death toll in today's s...,update 2 pm et the death toll in todays shooti...
